# Text recognition - テキスト認識

The next ready solution for image data that we will test is for detecting text. This can be used for both printed and handwritten text.

次に試す画像データ用のサービスはテキストの検出です。印刷した文章や手書きテキストに応用できます。

Let's start again by defining the environment variable for Google cloud authentication.

Googleクラウド認証用の環境変数を定義することから始めましょう。

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=gcp_credentials/mlpractice2020.json

Next load libraries that we will need.

次に必要なライブラリをインポートします。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageOps

from google.cloud import vision

## Recognition of handwritten text / 手書きテキストの認識

We will first try recognizing text that we ourselves manually write. 

We will use the canvas input that you already saw in previous sessions when we worked with the MNIST dataset. At that time we manually implemented the learning for detecting handwritten digits 0 to 9. Compared to that the service that GCP provides is better in many aspects:

- It can recognize letters, not just numbers. It also supports different languages and writting system (including kanji and japanese).

- It can detect whole words and sentences, instead of just one character.

- It uses state of the art machine learning solutions so it gives very good results out of the box (i.e. no need to adjust parameters etc.).

まずは、自分で書いた手書きテキストの認識してみましょう。

先日のMNISTデータセットのセッションですでに見たキャンバス入力ツールを使用します。当時、手書きの数字0から9を検出するための学習を手動で実装しました。それに比べて、GCPが提供するサービスは多くの点で優れています。

- 数字だけでなく文字も認識できます。それはまた異なった言語や文字をサポートしている（漢字・日本語も使えます）

- 1文字だけでなく、単語や文全体を検出できる

- 最先端の機械学習手法を使用しているので、そのままで（つまり、学習やパラメータ調整を何もしなくても）非常にいい結果が得られる

The following code is used to setup the canvas for writing. You do not need to understand the details of the code. It is enough just to know that every time something is written on the canvas the canvas content will be automatically saved inside a variable named `image` (as numpy array).

以下は書き込み用のキャンバスを設定するためのコードです。このコードの詳細を理解する必要はありません。キャンバスに何かが書かれるたびに、キャンバスの中身が`image`というnumpy配列に保存される、という仕組みになっています。

In [ ]:
import ipywidgets as widgets
import jupyter_drawing_pad as jd

out1 = widgets.Output(layout=widgets.Layout(width='400px'))
jdp = jd.CustomBox()

draw_pad = jdp.drawing_pad
clear_btn = jdp.children[1].children[1]

@out1.capture() 
def w1_CB(change):
    from scipy.signal import convolve2d
    from cv2 import resize, INTER_CUBIC
    
    global image

    data = change['new']
    if len(data[0]) > 2:
        # Format the canvas output to obtain a 28 x 28 image 
        x = np.array(data[0])
        y = np.array(data[1])
        
        # we plot stroke by stroke for 
        # assuming there is at least 200ms between each stroke 
        line_breaks = np.where(np.diff(np.array(data[2])) > 200)[0]
        # adding end of array
        line_breaks = np.append(line_breaks,len(data[2]))
        
        # Plot to canvas
        from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
        fig = plt.figure()
        canvas = FigureCanvas(fig)
        ax = fig.gca()

        # plot all strokes
        plt.plot(x[:line_breaks[0]],y[:line_breaks[0]],color='red',linewidth=4)
        for i in range(1,len(line_breaks)):
            plt.plot(x[line_breaks[i-1]+1:line_breaks[i]],y[line_breaks[i-1]+1:line_breaks[i]],color='red',linewidth=4)
        
        plt.xlim(0,460)
        plt.ylim(0,250)
        plt.axis("off")
        # plt.show()    
        
        canvas.draw()       # draw the canvas, cache the renderer

        # convert to numpy array - just for plotting
        imageflat = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
         # not sure why this size...
        image = np.reshape(imageflat,(288,432,3))
        # cut figure edges
        image = image[35:250,50:390]
        
        # draw again the converted image (not really necessary, but for testing)
        plt.clf()
        plt.imshow(image, aspect='equal', interpolation='none')
        plt.axis("off")
        plt.show()

        # Schedule for clearing
        out1.clear_output(wait=True)
    else:
        pass
        
    
draw_pad.observe(w1_CB, names='data')

hb = widgets.HBox([draw_pad, clear_btn, out1])
display(hb)

Try writing some text in the above canvas. Japanese, English and [multiple other languages](https://cloud.google.com/vision/docs/languages) can be used - try first the one which you are most comfortable writing.

上記のキャンバスにテキストを書いてみてください。日本語、英語や[その他の言語](https://cloud.google.com/vision/docs/languages)を使えます。最初は自分に最も書きやすい言語を使ってください。

The following code saves the content of the canvas into a file ("image_file.png"), which we will use later.

次のコードはキャンバスの内容をファイル"image_file.png"に保存します。このファイルは後で使います。

In [ ]:
im = Image.fromarray(image)
imagepath = "image_file.png"
im.save(imagepath,format="PNG")

Now let's load the image that we just saved an prepare it for sending to Google Cloud. (This part of the code is the same as we used for face detection.)

それでは、保存した画像をロードして、Google Cloudに送信するための準備をしましょう。 （このコードは顔検出に使用したコードと同じです。）

In [ ]:
client = vision.ImageAnnotatorClient()

with open(imagepath, 'rb') as image_file:
    content = image_file.read()

image = vision.types.Image(content=content)

And let's carry out the actual detection.

そして、実際の検出を実行しましょう。

In [ ]:
response = client.document_text_detection(image=image)

# try using this version if you are getting wrong results and want to give a hint to the detection program which language to use
# response = client.document_text_detection(image=image, image_context={"language_hints": ["ja"]})

The variable  `response` contains the detection result. The part `full_text_annotation` gives the whole detected text.

変数`response`には検出結果が含まれています。`full_text_annotation`の部分には、検出されたテキスト全体が入っています。

In [ ]:
print(response.full_text_annotation.text)

Try changing the text in the canvas and running the detection again.

キャンバス内のテキストを変更して、検出をもう一度実行してみてください。

## Recognition of MNIST images / MNIST画像の認識

In order to compare with results from the previous sessions let's also try to see if GCP can recognize the numbers on the MNIST dataset images.

Note that now we will not have to train a neural network or some other machine learning algorithm by ourselves. We can just ask the cloud service to recognize which number is in the image.

先週や先々週の結果と比較するために、GCPがMNISTデータセット画像の番号を認識できるかどうかも確認してみましょう。

なお、前のセッションと違って、ニューラルネットワーク等の機械学習アルゴリズムを自分で訓練する必要は全くありません。どの数字が画像内にあるかを認識するようにクラウドサービスに依頼するだけです。

Let's see how well the cloud service performs. The following code loads 5 randomly selected images from the MNIST dataset and makes a combines them to get a single image (you will get a different image each time you execute the code).

クラウドサービスがどの程度うまく機能するかを見てみましょう。 次のコードは、MNISTデータセットからランダムに選択された5つの画像をロードし、それらを組み合わせて単一の画像を取得します（コードを実行するたびに異なる画像が取得されます）。

In [ ]:
from mnist_loader import MNISTImageLoader
# initialize and randomize
mnist_image_loader = MNISTImageLoader(np.random.randint(10000))
# load just 1 sample
X, y = mnist_image_loader.samples(5)
# stack 5 images together
X = np.hstack((X[0,:,:,0],X[1,:,:,0],X[2,:,:,0],X[3,:,:,0],X[4,:,:,0]))

# plot image and print number
plt.imshow(X) 
plt.axis('off')
print("Loaded image has numbers: " + str(y))

The following code saves the content of the canvas into a file ("imageX.png"), which we later send to the Google Cloud.

次のコードは上の画像の内容をファイル"imageX.png"に保存します。このファイルは後でGoogleクラウドに送信します。

In [ ]:
imX = Image.fromarray(X)

# the MNIST images are very small, we need to resize them to get a good detection result from GCP
# we also invert the colors
#imX=ImageOps.invert(imX.resize((5*140,140))) # 5x resized
imX=ImageOps.invert(imX)

imagepathX = "imageX.png"
imX.save(imagepathX,format="PNG")

Now let's load the image that we just saved an prepare it for sending to Google Cloud. (This part of the code is the same as we used for face detection.)

ただいま保存した画像をロードし、Google Cloudに送信する準備をしましょう。（このコードの部分は、顔検出に使用したものと同様です。）

In [ ]:
client = vision.ImageAnnotatorClient()

with open(imagepathX, 'rb') as image_file:
    content = image_file.read()

image = vision.types.Image(content=content)

Finally, we run the actual detection.

それでは、実際の検出を実行します。

In [ ]:
response = client.document_text_detection(image=image, image_context={"language_hints": ["en"]})

The variable  `response` contains the detection result. The part `full_text_annotation` gives the whole detected text.

In [ ]:
print(response.full_text_annotation.text)

Did you get the correct result?
<br>
(The text detection is not specialised for detecting digits, so it may not always work well.)

正しい結果が出ましたか？
<br>
（テキスト認識は数字の認識に特化していないため、常にうまく機能するとは限りません）

---
## Try  it yourself ! / 自分で試そう！

When you are ready open the [notebook with exercises](session2-playground.ipynb).

それでは、[プレイグランド](session2-playground.ipynb)を開きましょう。